In [1]:
import pandas as pd 
import geopandas as gpd

In [2]:
data=gpd.read_file("../00_source_data/CollegeUniversityCampuses/CollegeUniversityCampuses.shp")

In [7]:
data

,UNIQUEID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,STATUS,...,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,TOT_ENROLL,TOT_EMP,SHELTERID,geometry,geo_type
0,105367,OTTAWA UNIVERSITY-PHOENIX,9414 NORTH 25TH AVENUE,PHOENIX,AZ,85021,NOT AVAILABLE,(602) 371-1188,2,A,...,www.ottawa.edu,2018-02-28,IMAGERY/OTHER,2019-01-24,www.ottawa.edu,809,114,NOT AVAILABLE,"POLYGON ((-12480419.928 3971527.354, -12480311...",POLYGON
1,110361,CALIFORNIA BAPTIST UNIVERSITY,8432 MAGNOLIA AVE,RIVERSIDE,CA,92504,3297,(877) 228-3615,2,A,...,https://calbaptist.edu/map,2018-05-10,IMAGERY/OTHER,2019-01-24,www.calbaptist.edu,10486,1656,NOT AVAILABLE,"POLYGON ((-13072418.831 4019240.575, -13071986...",POLYGON
2,111391,CALIFORNIA WESTERN SCHOOL OF LAW,225 CEDAR ST,SAN DIEGO,CA,92101,3046,(619) 239-0391,2,A,...,https://www.cwsl.edu,2018-05-11,IMAGERY/OTHER,2019-01-24,https://www.cwsl.edu,802,219,NOT AVAILABLE,"POLYGON ((-13042517.805 3858457.172, -13042435...",POLYGON
3,113698,DOMINICAN UNIVERSITY OF CALIFORNIA,50 ACACIA AVE,SAN RAFAEL,CA,94901,2298,(415) 457-4440,2,A,...,https://www.dominican.edu/about/campus/mapsand...,2018-08-30,IMAGERY/OTHER,2019-01-24,www.dominican.edu,1779,616,NOT AVAILABLE,"MULTIPOLYGON (((-13637570.792 4576797.967, -13...",MULTIPOLYGON
4,192563,LIU BRENTWOOD,"MICHAEL J. GRANT CAMPUS, SUFFOLK COUNTY COMMUN...",BRENTWOOD,NY,11717,NOT AVAILABLE,(631) 287-8500,2,C,...,http://www.liu.edu/Brentwood/About/Visiting/Ca...,2018-12-19,IMAGERY/OTHER,2019-04-11,liu.edu/Brentwood,-999,-999,NOT AVAILABLE,"POLYGON ((-8156290.559 4982259.419, -8156297.2...",POLYGON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6049,11493801,GAVILAN COLLEGE - HOLLISTER SITE,365 FOURTH STREET,HOLLISTER,CA,95023,NOT AVAILABLE,NOT AVAILABLE,1,A,...,https://www.gavilan.edu/about/hollister.php,2018-12-19,IMAGERY/OTHER,2019-02-12,http://www.gavilan.edu/hollister/index.html,1361,-999,NOT AVAILABLE,"POLYGON ((-13514524.423 4418567.125, -13514465...",POLYGON
6050,11493802,GAVILAN COLLEGE - MORGAN HILL CAMPUS,17060 MONTEREY ST.,MORGAN HILL,CA,95037,NOT AVAILABLE,NOT AVAILABLE,1,A,...,https://www.gavilan.edu/about/morgan_hill.php,2018-12-19,IMAGERY/OTHER,2019-02-12,http://www.gavilan.edu/maps/index.html,1323,-999,NOT AVAILABLE,"POLYGON ((-13542122.174 4456753.673, -13542009...",POLYGON
6051,490805,PURDUE UNIVERSITY NORTHWEST,2200 169TH STREET,HAMMOND,IN,46323,NOT AVAILABLE,(219) 989-2400,1,A,...,https://www.pnw.edu/getting-to-pnw/visiting/dr...,2020-07-01,IMAGERY/OTHER,2020-07-01,www.pnw.edu,10473,1410,NOT AVAILABLE,"POLYGON ((-9737635.902 5098433.533, -9737487.6...",POLYGON
6052,195562,ST. JOSEPH'S COLLEGE-LONG ISLAND,155 W ROE BLVD,PATCHOGUE,NY,11772,NOT AVAILABLE,(631) 687-5100,2,A,...,https://www.sjcny.edu/long-island/about/campus...,2020-07-01,IMAGERY/OTHER,2020-07-01,sjcny.edu,4115,761,NOT AVAILABLE,"POLYGON ((-8129403.527 4979159.269, -8129426.8...",POLYGON


In [4]:

for i, r in data.iterrows():
    if "MULTIPOLYGON" in str(data.loc[i,'geometry']):
        data.loc[i,'geo_type']='MULTIPOLYGON'
    elif "POLYGON" in str(data.loc[i,'geometry']):
        data.loc[i,'geo_type']='POLYGON'
    elif "POINT" in str(data.loc[i,'geometry']):
        data.loc[i,'geo_type']='POINT'
        



In [5]:
data['geo_type'].value_counts()

POLYGON         5307
MULTIPOLYGON     747
Name: geo_type, dtype: int64

In [11]:
type(data)

geopandas.geodataframe.GeoDataFrame

In [5]:
def string_punc(test_str):

    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

    try:

        test_str=str(test_str).lower()
        #print(test_str)
        for ele in test_str:
            if ele in punc:
                test_str = test_str.replace(ele, "")
        processed_lst=[]
        for word in test_str.split():
            if word not in ['the','at','in','and','of']:
                processed_lst.append(word)

        test_str=' '.join(processed_lst)
        print(test_str)
        return test_str.lower()
    except:
        return 'Null'

In [6]:
data['preprocessed_name'] = data['NAME'].apply(string_punc)

ottawa universityphoenix
california baptist university
california western school law
dominican university california
liu brentwood
argosy universitythe art institute californiasan francisco
point loma nazarene university
intercoast collegesfairfield
st vincents college
rasmussen collegeflorida
hawaii pacific university
flint hills technical college
lincoln college technologymelrose park
university phoenixoklahoma
columbia college chicago
jefferson community technical college
sumner college
santa barbara business college
tricommunity adult education
georgia military college madison
lincoln college technologycolumbia
california coast university
toni guy hairdressing academyshoreline
richport technical college
university phoenixidaho
oikos university
four rivers career center
north shore community college
vista collegeonline
williamson college trades
berkowits school electrolysis
international college holistic studies
no grease barber school
purdue university globallewiston
georgia state 

northern michigan university
northwestern michigan college
michigan technological university
university michigandearborn
university michiganflint
mid michigan college
monroe county community college
montcalm community college
northwood university
oakland university
olivet college
sacred heart major seminary
saginaw valley state university
siena heights university
st clair county community college
southwestern michigan college
specs howard school media arts
spring arbor university
finlandia university
walsh college
washtenaw community college
central lakes collegebrainerd
wayne state university
western michigan university
alexandria technical community college
anoka technical college
anokaramsey community college
dunwoody college technology
riverland community college
northwest technical college
bemidji state university
bethany lutheran college
bethel university
carleton college
concordia college moorhead
concordia universitysaint paul
dakota county technical college
martin luther colle

brightwood collegeindianapolis
suny college oswego
university phoenixmissouri
sacramento city college
yeshivath viznitz
youngstown state university
suny empire state college
sullivan county community college
samuel merritt university
middlebury institute international studies monterey
ithaca college
bethunecookman university
suny westchester community college
keuka college
cbd college
pacific college oriental medicinesan diego
canisius college
south baylo university
health style institute
marist college
lassen community college
marion technical college
uei collegegardena
albany medical college
marshall b ketchum university
lincoln university
new england tractor trailer training school rhode island
herzing universitykenner
empire college
clarkson university
barry university
palau community college
cuny bronx community college
dorsey business schoolsroseville
ohio state universitymarion campus
wright state universitylake campus
universidad ana g mendezgurabo campus
university phoenixmich

colorado mountain college rifle
front range community college boulder county campus
front range community college larimer campus
colorado mesa university montrose campus
pueblo community college fremont campus
red rocks community college arvada campus
trinidad state junior college valley campus
quinnipiac university york hill campus
quinnipiac university north haven campus
university district columbia community college
yale university west campus
university delaware carver reseach education center campus
wilmington university wilson graduate center campus
wilmington university dover campus
wilmington university rehoboth campus
wilmington university georgetown campus
university district columbia community college bertie brackus campus
george washington university mt vernon college
university florida martin co ifas ext
wesley theological seminary wesley downtown campus
florida atlantic university treasure coast
florida technical college kissimmee campus
florida technical college deland c

In [12]:
#Extracting the centroid from each college polygon
data['centroid']=data.centroid
data = data.assign(x_centroid = lambda x: (x['centroid'].x))
data = data.assign(y_centroid = lambda x: (x['centroid'].y))
data['centroid_geom'] = data['x_centroid'].map(str) + ', ' + data['y_centroid'].map(str)




In [13]:
data

,UNIQUEID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,STATUS,...,WEBSITE,TOT_ENROLL,TOT_EMP,SHELTERID,geometry,preprocessed_name,centroid,x_centroid,y_centroid,centroid_geom
0,105367,OTTAWA UNIVERSITY-PHOENIX,9414 NORTH 25TH AVENUE,PHOENIX,AZ,85021,NOT AVAILABLE,(602) 371-1188,2,A,...,www.ottawa.edu,809,114,NOT AVAILABLE,"POLYGON ((-12480419.928 3971527.354, -12480311...",ottawa universityphoenix,POINT (-12480368.112 3971492.550),-1.248037e+07,3.971493e+06,"-12480368.111719692, 3971492.550103365"
1,110361,CALIFORNIA BAPTIST UNIVERSITY,8432 MAGNOLIA AVE,RIVERSIDE,CA,92504,3297,(877) 228-3615,2,A,...,www.calbaptist.edu,10486,1656,NOT AVAILABLE,"POLYGON ((-13072418.831 4019240.575, -13071986...",california baptist university,POINT (-13071805.749 4019121.787),-1.307181e+07,4.019122e+06,"-13071805.749257807, 4019121.7871760847"
2,111391,CALIFORNIA WESTERN SCHOOL OF LAW,225 CEDAR ST,SAN DIEGO,CA,92101,3046,(619) 239-0391,2,A,...,https://www.cwsl.edu,802,219,NOT AVAILABLE,"POLYGON ((-13042517.805 3858457.172, -13042435...",california western school law,POINT (-13042476.335 3858426.366),-1.304248e+07,3.858426e+06,"-13042476.334521357, 3858426.3664698643"
3,113698,DOMINICAN UNIVERSITY OF CALIFORNIA,50 ACACIA AVE,SAN RAFAEL,CA,94901,2298,(415) 457-4440,2,A,...,www.dominican.edu,1779,616,NOT AVAILABLE,"MULTIPOLYGON (((-13637570.792 4576797.967, -13...",dominican university california,POINT (-13638208.049 4576613.670),-1.363821e+07,4.576614e+06,"-13638208.049442334, 4576613.669529073"
4,192563,LIU BRENTWOOD,"MICHAEL J. GRANT CAMPUS, SUFFOLK COUNTY COMMUN...",BRENTWOOD,NY,11717,NOT AVAILABLE,(631) 287-8500,2,C,...,liu.edu/Brentwood,-999,-999,NOT AVAILABLE,"POLYGON ((-8156290.559 4982259.419, -8156297.2...",liu brentwood,POINT (-8156683.197 4982572.374),-8.156683e+06,4.982572e+06,"-8156683.196948491, 4982572.373837653"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6049,11493801,GAVILAN COLLEGE - HOLLISTER SITE,365 FOURTH STREET,HOLLISTER,CA,95023,NOT AVAILABLE,NOT AVAILABLE,1,A,...,http://www.gavilan.edu/hollister/index.html,1361,-999,NOT AVAILABLE,"POLYGON ((-13514524.423 4418567.125, -13514465...",gavilan college hollister site,POINT (-13514495.919 4418529.515),-1.351450e+07,4.418530e+06,"-13514495.91943143, 4418529.515234517"
6050,11493802,GAVILAN COLLEGE - MORGAN HILL CAMPUS,17060 MONTEREY ST.,MORGAN HILL,CA,95037,NOT AVAILABLE,NOT AVAILABLE,1,A,...,http://www.gavilan.edu/maps/index.html,1323,-999,NOT AVAILABLE,"POLYGON ((-13542122.174 4456753.673, -13542009...",gavilan college morgan hill campus,POINT (-13541961.062 4456732.530),-1.354196e+07,4.456733e+06,"-13541961.06207996, 4456732.529931318"
6051,490805,PURDUE UNIVERSITY NORTHWEST,2200 169TH STREET,HAMMOND,IN,46323,NOT AVAILABLE,(219) 989-2400,1,A,...,www.pnw.edu,10473,1410,NOT AVAILABLE,"POLYGON ((-9737635.902 5098433.533, -9737487.6...",purdue university northwest,POINT (-9737562.887 5098318.702),-9.737563e+06,5.098319e+06,"-9737562.887115747, 5098318.702358599"
6052,195562,ST. JOSEPH'S COLLEGE-LONG ISLAND,155 W ROE BLVD,PATCHOGUE,NY,11772,NOT AVAILABLE,(631) 687-5100,2,A,...,sjcny.edu,4115,761,NOT AVAILABLE,"POLYGON ((-8129403.527 4979159.269, -8129426.8...",st josephs collegelong island,POINT (-8129200.688 4979371.714),-8.129201e+06,4.979372e+06,"-8129200.687651995, 4979371.713513117"


In [7]:
master_df = pd.read_csv("../00_source_data/College_Data/SLSV Master Campus Sheet - Master Sheet.csv", header=1)
print("Original Master:",len(master_df))
master_df['preprocessed_name'] = master_df['School Name'].apply(string_punc)
master_df=master_df[~master_df['preprocessed_name'].str.contains('policy|law|phd|graduate|grad|health|medicine|medical|music|business|nursing|film|art|arts|design|commerce', regex=True)]
print("Length of Master Table:",len(master_df))


Original Master: 3114
alaska christian college
alaska pacific university
ilisagvik college
prince william sound community college
university alaska anchorage
university alaska fairbanks
university alaska southeast
alabama a m university
alabama state university
amridge university
athens state university
auburn university
auburn university montgomery
bevill state community college
birmingham southern college
bishop state community college
calhoun community college
central alabama community college
chattahoochee valley community college
columbia southern university
concordia college alabama
enterprise state community college
faulkner university
gadsden state community college
george c wallace community college
heritage christian university
herzing university birmingham
huntingdon college
jacksonville state university
jefferson state community college
jf drake state technical college
judson college
lawson state community college
lurleen b wallace community college
marion military institut

university new mexicotaos campus
university new mexicovalencia county campus
university southwest
western new mexico university
college southern nevada
great basin college
nevada state college
northwest career college
roseman university health sciences
roseman university health sciences south jordan
sierra nevada college
touro university
truckee meadows community college
university nevada las vegas
university nevada reno
western nevada college
adelphi university
adirondack community college
albany college pharmacy health sciences
albany law school
albany medical college
alfred university
american academy mcallister institute
asa college
bank street college education
bard college
barnard college
berkeley college new york
boricua college
brooklyn law school
canisius college
cazenovia college
clarkson university
clarkson university capital region campus
colgate university
college mount saint vincent
college new rochelle
college saint rose
columbia university
columbiagreene community colle

In [8]:
final_data=master_df.merge(data, left_on='preprocessed_name', right_on='preprocessed_name',how='inner')




In [9]:
final_data

,State,OPE ID,IPED ID,School Name,Institution Type,# of Nat'l AP Partners,# of Partners,Civic Holiday,MSI,NSLVE,...,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,TOT_ENROLL,TOT_EMP,SHELTERID,geometry
0,AK,1061.00,102669.00,Alaska Pacific University,"Private, 4 Year",0.0,1.0,1.0,ANNH,No,...,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://www.alaskapacific.edu/visit/campus-map/,2017-11-28,IMAGERY/OTHER,2017-11-28,www.alaskapacific.edu,502,148,NOT AVAILABLE,"POLYGON ((-16676514.708 8669707.202, -16676371..."
1,AK,11462.00,102553.00,"University of Alaska, Anchorage","Public, 4 Year",1.0,4.0,2.0,,Yes,...,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://www.uaa.alaska.edu/map/,2017-11-27,IMAGERY/OTHER,2017-11-27,www.uaa.alaska.edu,14644,2442,10822636,"MULTIPOLYGON (((-16676651.405 8670483.062, -16..."
2,AK,1063.00,102614.00,"University of Alaska, Fairbanks","Public, 4 Year",0.0,1.0,0.0,ANNH,Yes,...,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://www.uaf.edu/files/fs/bristolbaymasterp...,2017-11-27,IMAGERY/OTHER,2019-02-15,www.uaf.edu,7404,2467,NOT AVAILABLE,"POLYGON ((-16457906.644 9572524.920, -16457895..."
3,AK,1065.00,102632.00,"University of Alaska, Southeast","Public, 4 Year",0.0,2.0,0.0,ANNH & NASNTI,No,...,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",http://www.uas.alaska.edu/tours/campus-juneau....,2017-11-27,IMAGERY/OTHER,2017-11-27,www.uas.alaska.edu,2266,405,NOT AVAILABLE,"MULTIPOLYGON (((-14987767.560 8049741.470, -14..."
4,AL,1002.00,100654.00,Alabama A & M University,"Public, 4 Year",3.0,5.0,0.0,HBCU,Yes,...,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",http://www.aamu.edu/information/Pages/CampusMa...,2018-03-05,IMAGERY/OTHER,2018-03-05,www.aamu.edu/,6106,977,NOT AVAILABLE,"POLYGON ((-9634964.878 4134743.399, -9635341.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,WY,9259.00,240620.00,Laramie County Community College,"Public, 2 Year",0.0,0.0,0.0,,No,...,JUNIOR COLLEGES,http://www.lccc.wy.edu/about/maps/index.aspx,2018-04-18,IMAGERY/OTHER,2018-04-18,www.lccc.wy.edu,4097,657,NOT AVAILABLE,"POLYGON ((-11663340.157 5028171.114, -11663340..."
2139,WY,3930.00,240666.00,Northern Wyoming Community College District,NaN,0.0,0.0,0.0,,No,...,JUNIOR COLLEGES,www.sheridan.edu,2018-04-18,IMAGERY/OTHER,2018-04-18,www.sheridan.edu,4168,549,NOT AVAILABLE,"MULTIPOLYGON (((-11904254.778 5584446.352, -11..."
2140,WY,3931.00,240657.00,Northwest College,"Public, 2 Year",0.0,0.0,0.0,,No,...,JUNIOR COLLEGES,nwc.edu/about/campus,2018-04-18,IMAGERY/OTHER,2018-04-18,www.nwc.edu,1488,269,NOT AVAILABLE,"POLYGON ((-12108200.478 5584521.955, -12107404..."
2141,WY,3932.00,240727.00,University of Wyoming,"Public, 4 Year",1.0,3.0,0.0,,Yes,...,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",http://www.uwyo.edu/foundation/gateway-center/...,2018-04-18,IMAGERY/OTHER,2018-04-18,www.uwyo.edu,12450,2795,NOT AVAILABLE,"MULTIPOLYGON (((-11749205.381 5059753.412, -11..."


In [14]:
 _=data.to_file("R1.geojson", driver='GeoJSON')

TypeError: Cannot interpret '<geopandas.array.GeometryDtype object at 0x17db1afa0>' as a data type